In [1]:
import sys, os, time, pickle, h5py, logging, warnings
from tqdm import tqdm

import numpy as np
from scipy import interpolate,integrate

import jax
import jax.numpy as jnp

# The LambdaCDM cosmology
from astropy.cosmology import Planck18 as cosmo
from astropy import cosmology, constants, units

# Import modified 21cmFAST
import py21cmfast as p21c
from py21cmfast import cache_tools

# Configure environment for use with DarkHistory
os.environ['DH_DIR']='/global/scratch/projects/pc_heptheory/fosterjw/21CM_Project/DarkHistory/'
os.environ['DM21CM_DIR'] = '../'
os.environ['DM21CM_DATA_DIR'] = '../..'
sys.path.append(os.environ['DH_DIR'])
from darkhistory.spec.spectrum import Spectrum # use branch test_dm21cm

# Import DM21CM code for this project
sys.path.append("..")
import dm21cm.physics as phys
from dm21cm.dm_params import DMParams
from dm21cm.utils import load_dict
from dm21cm.data_loader import load_data
from dm21cm.evolve import evolve
from dm21cm.field_smoother import WindowedData

# Plotting
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib import colormaps as cms
mpl.rc_file("../matplotlibrc")
mpl.rcParams['text.usetex']=False
# Logging
logging.getLogger().setLevel(logging.INFO)
logging.getLogger('21cmFAST').setLevel(logging.CRITICAL+1)
logging.getLogger('py21cmfast._utils').setLevel(logging.CRITICAL+1)
logging.getLogger('py21cmfast.wrapper').setLevel(logging.CRITICAL+1)
logging.info(f'Using 21cmFAST version {p21c.__version__}')

import powerbox

/global/scratch/projects/pc_heptheory/fosterjw/21CM_Project/21cmFAST/src/py21cmfast/_cfg.py:58: UserWarning: Your configuration file is out of date. Updating...
  warnings.warn(
/global/scratch/projects/pc_heptheory/fosterjw/21CM_Project/21cmFAST/src/py21cmfast/_cfg.py:42: UserWarning: Your configuration file is out of date. Updating...
  warnings.warn("Your configuration file is out of date. Updating...")
INFO:root:Using 21cmFAST version 0.1.dev1579+g6b1da6d
INFO:root:Using 21cmFAST version 0.1.dev1579+g6b1da6d


# These are some helper functions

In [2]:
def get_z_edges(z_start, z_end, z_step_factor):
    
    z_list = [z_start]

    while z_list[-1] > z_end:
        z_list.append((z_list[-1] + 1) / z_step_factor-1)
    return z_list

def split_xray(phot_N, ix_lo, ix_hi):
    """Split a photon spectrum (N in bin) into bath and xray band."""
    bath_N = np.array(phot_N).copy()
    xray_N = np.array(phot_N).copy()
    bath_N[ix_lo:ix_hi] *= 0
    xray_N[:ix_lo] *= 0
    xray_N[ix_hi:] *= 0
    
    return bath_N, xray_N

# Configure Our 21cmFAST Run

In [3]:
p21c.config['direc'] = '../DebugCache/'
cache_tools.clear_cache()

# The range of times and how we step
z_start = 44
z_end = 6.
z_step_factor = 1.01

# The size and resolution of our box
HII_DIM = 50 # ['Dimensionless']
BOX_LEN = 100 # ['Mpccm']


# Our energy injection model
dm_params = DMParams(mode = 'swave', primary = 'phot_delta',
                     m_DM = 1e10, sigmav = 1e-23)

struct_boost_model = 'erfc 1e-3'
enable_elec = False
force_reload_tf = False
run_mode = 'xray'
dhinit_list = ['phot', 'T_k', 'x_e']
dhtf_version = '230629'

In [4]:
# Determine the appropriate boost factor
if dm_params.mode == 'swave':
    struct_boost = lambda rs: phys.struct_boost_func(model=struct_boost_model)(rs)
else:
    struct_boost = lambda rs: 1

In [5]:
p21c.global_params.Z_HEAT_MAX = z_start
p21c.global_params.ZPRIME_STEP_FACTOR = z_step_factor
p21c.global_params.CLUMPING_FACTOR = 1.

p21c_initial_conditions = p21c.initial_conditions(
    user_params = p21c.UserParams(
        HII_DIM = HII_DIM,
        BOX_LEN = BOX_LEN,
        N_THREADS = 32,
    ),
    cosmo_params = p21c.CosmoParams(
        OMm = cosmo.Om(0),
        OMb = cosmo.Ob(0),
        POWER_INDEX =cosmo.meta['n'],
        SIGMA_8 = cosmo.meta['sigma8'],
        hlittle = cosmo.h,
    ),
    random_seed = 54321,
    write = True,
)

# Some details regarding our stepping
z_edges = get_z_edges(z_start, z_end, 1.01)

/global/scratch/projects/pc_heptheory/fosterjw/21CM_Project/21cmFAST/src/py21cmfast/inputs.py:487: UserWarning: The USE_INTERPOLATION_TABLES setting has changed in v3.1.2 to be default True. You can likely ignore this warning, but if you relied onhaving USE_INTERPOLATION_TABLES=False by *default*, please set it explicitly. To silence this warning, set it explicitly to True. Thiswarning will be removed in v4.
  warnings.warn(


In [6]:
def p21_step(z_eval, perturbed_field, spin_temp, ionized_box,
             input_heating = None, input_ionization = None, input_jalpha = None):
    
    # Calculate the spin temperature, possibly using our inputs
    spin_temp = p21c.spin_temperature(perturbed_field=perturbed_field,
                                      previous_spin_temp = spin_temp,
                                      input_heating_box = input_heating,
                                      input_ionization_box = input_ionization,
                                      input_jalpha_box = input_jalpha)
    
    # Calculate the ionized box
    ionized_box = p21c.ionize_box(perturbed_field = perturbed_field,
                                  previous_ionize_box=ionized_box,
                                  spin_temp=spin_temp)
    
    
    # Calculate the brightness temperature
    brightness_temp = p21c.brightness_temperature(ionized_box=ionized_box,
                                                  perturbed_field = perturbed_field,
                                                  spin_temp = spin_temp)
    
    # Construct a coevaluation set out of this data
    coeval = p21c.Coeval(redshift = z_eval, 
                         initial_conditions = p21c_initial_conditions,
                         perturbed_field = perturbed_field,
                         ionized_box = ionized_box,
                         brightness_temp = brightness_temp,
                         ts_box = spin_temp)
    
    # Now return the spin temperature and ionized box because we will need them later
    return spin_temp, ionized_box, brightness_temp

def gen_injection_boxes(next_z, p21c_initial_conditions):
    
    # Instantiate the injection arrays
    input_heating = p21c.input_heating(redshift=next_z, init_boxes=p21c_initial_conditions, write=False)
    input_ionization = p21c.input_ionization(redshift=next_z, init_boxes=p21c_initial_conditions, write=False)
    input_jalpha = p21c.input_jalpha(redshift=next_z, init_boxes=p21c_initial_conditions, write=False)
    
    return input_heating, input_ionization, input_jalpha

# Dark History Interface Class

In [7]:
class DarkHistoryWrapper:
    
    def __init__(self, HII_DIM, dhinit_list, z_step_factor, enable_elec = False):
        
        # Basic run parameters
        self.HII_DIM = HII_DIM
        self.norm = 1/HII_DIM**3
        self.enable_elec = enable_elec

        
        # Setting up the DH Transfer functions
        self.set_abcissa(z_step_factor)
        self.set_transfers(dhinit_list)
        
        # The DH initial condition that we will match
        dhinit_fn = f"{p21c.config['direc']}/dhinit_soln.p"
        self.dhinit_soln = pickle.load(open(dhinit_fn, 'rb'))
    
    def set_abcissa(self, z_step_factor):
        
        '''
        This sets the abcissa for the photon energies and electron energies. We also
        set the size out the last dimension of the deposition box.
        '''
        
        abscs = load_dict(os.environ['DM21CM_DIR'] + f"/data/abscissas/abscs_{dhtf_version}.h5")
        
        if not np.isclose(np.log(z_step_factor), abscs['dlnz']):
            raise ValueError('zplusone_step_factor and dhtf_version mismatch')
            
        self.photeng = abscs['photE']
        self.eleceng = abscs['elecEk']
        self.dep_size = len(abscs['dep_c'])
            
    def set_transfers(self, dhinit_list):
        '''
        This function initializes the DH transfer functions used in the 21cm run. 
        '''
        
        data_prefix = os.environ['DM21CM_DATA_DIR'] + f'/tf/{dhtf_version}/phot'

        
        # Initialize photon transfer functions
        self.phot_prop_tf = load_data('phot_prop', prefix=data_prefix, reload=force_reload_tf)
        self.phot_scat_tf = load_data('phot_scat', prefix=data_prefix, reload=force_reload_tf)
        self.phot_dep_tf = load_data('phot_dep', prefix=data_prefix, reload=force_reload_tf)
        self.phot_prop_tf.set_fixed_in_spec(dm_params.inj_phot_spec.N)
        self.phot_scat_tf.set_fixed_in_spec(dm_params.inj_phot_spec.N)
        self.phot_dep_tf.set_fixed_in_spec(dm_params.inj_phot_spec.N)
    
        # Initialize electron transfer functions if desired
        if self.enable_elec:
            raise NotImplementedError
            self.elec_phot_tf = load_data('elec_phot', prefix=data_prefix, reload=force_reload_tf)
            self.elec_dep_tf = load_data('elec_dep', prefix=data_prefix, reload=force_reload_tf)
            self.elec_phot_tf.set_fixed_in_spec(dm_params.inj_elec_spec.N)
            self.elec_dep_tf.set_fixed_in_spec(dm_params.inj_elec_spec.N)
            
    def set_empty_arrays(self):
        '''
        This is a convenience method that sets the properly shaped deposition box for 
        use in our evolution
        '''
            
        # Get the empty spectrum arrays
        self.prop_phot_N = np.zeros_like(self.photeng) # [N / Bavg]
        self.emit_phot_N = np.zeros_like(self.photeng) # [N / Bavg]

        # Get the empty deposition box. ('H ion', 'He ion', 'exc', 'heat', 'cont', 'xray')        
        self.dep_box = np.zeros((self.HII_DIM, self.HII_DIM, self.HII_DIM, self.dep_size))
        
        # We will also set the tf_kwargs to None to raise an error if they have not been reset
        self.tf_kwargs = None
        
    def set_tf_kwargs(self, tf_kwargs):
        '''
        Set the tf_kwargs
        '''
        
        self.tf_kwargs = tf_kwargs

    def get_state_arrays(self):

        '''
        This is a convenience method that provides the properly shaped deposition box for 
        use in our evolution
        '''
        
        # Return these
        return self.prop_phot_N, self.emit_phot_N, self.dep_box
    
    def photon_injection(self, spec_object, bath = True, weight_box = None, ots = False):
        if bath:
            self.prop_phot_N += self.norm*self.phot_prop_tf(in_spec=spec_object.N, sum_result=True, **tf_kwargs) # [N / Bavg]
            weight_box = np.ones((self.HII_DIM, self.HII_DIM, self.HII_DIM)) # dummy
            
        if ots:
            self.emit_phot_N += self.norm*self.phot_prop_tf(in_spec = spec_object.N, sum_result = True, sum_weight = weight_box.ravel(), **self.tf_kwargs)
            
        self.emit_phot_N += self.norm*self.phot_scat_tf(in_spec=spec_object.N, sum_result=True, sum_weight = weight_box.ravel(), **self.tf_kwargs)
        self.dep_box += weight_box[..., None] * self.phot_dep_tf(in_spec=spec_object.N, sum_result=False, **self.tf_kwargs).reshape(self.dep_box.shape) # [eV / Bavg]
     
        #return prop_phot_N, emit_phot_N, dep_box

In [8]:
dh_wrapper = DarkHistoryWrapper(HII_DIM, dhinit_list, 1.01)

INFO:root:Loaded photon propagation transfer function.
INFO:root:Loaded photon scattering transfer function.
INFO:root:Loaded photon deposition transfer function.
INFO:jax._src.xla_bridge:Unable to initialize backend 'cuda': module 'jaxlib.xla_extension' has no attribute 'GpuAllocatorConfig'
INFO:jax._src.xla_bridge:Unable to initialize backend 'rocm': module 'jaxlib.xla_extension' has no attribute 'GpuAllocatorConfig'
INFO:jax._src.xla_bridge:Unable to initialize backend 'tpu': module 'jaxlib.xla_extension' has no attribute 'get_tpu_client'


# Field Smoother for Propagating X-Rays

In [9]:
ex_lo, ex_hi = 1e2, 1e4 # [eV]
ix_lo = np.searchsorted(dh_wrapper.photeng, ex_lo) # i of first bin greater than ex_lo, excluded
ix_hi = np.searchsorted(dh_wrapper.photeng, ex_hi) # i of first bin greater than ex_hi, included

attenuation_arr = np.ones((len(z_edges), len(dh_wrapper.photeng))) # same redshift locations as z_mids

xray_fn = p21c.config['direc']+'/xray_brightness.h5'
if os.path.isfile(xray_fn):
    os.remove(xray_fn)
    
xray_windowed_data = WindowedData(data_path = xray_fn, cosmo = cosmo, N = HII_DIM,
                                  dx = BOX_LEN / HII_DIM, cache = True)

# Evolution Loop

In [10]:
def get_time_step(i_z):
    current_z=z_edges[i_z]
    next_z = z_edges[i_z+1]
    
    # The cosmic time step size in s
    dt= ( cosmo.age(next_z) - cosmo.age(current_z) ).to('s').value
    return current_z, next_z, dt

cache_tools.clear_cache()
run_mode = 'xray'
xray_loop_start = 0

## Synchronize

In [11]:
# Initial step with no injection
perturbed_field = p21c.perturb_field(redshift=z_edges[0], init_boxes=p21c_initial_conditions)
spin_temp, ionized_box, brightness_temp = p21_step(z_edges[0], perturbed_field, None, None)

# Manual sychronization of 21cmFAST with DarkHistory state
if 'T_k' in dhinit_list:
    T_k_DH = np.interp(spin_temp.redshift, dh_wrapper.dhinit_soln['rs'][::-1] - 1,
                       dh_wrapper.dhinit_soln['Tm'][::-1] / phys.kB) # [K]
    spin_temp.Tk_box += T_k_DH - np.mean(spin_temp.Tk_box)

if 'x_e' in dhinit_list:
    x_e_DH = np.interp(spin_temp.redshift, dh_wrapper.dhinit_soln['rs'][::-1] - 1,
                       dh_wrapper.dhinit_soln['x'][::-1, 0]) # HI
    
    spin_temp.x_e_box += x_e_DH - np.mean(spin_temp.x_e_box)
    x_H_DH = 1 - x_e_DH
    ionized_box.xH_box += x_H_DH - np.mean(ionized_box.xH_box)

if 'phot' in dhinit_list:
    logrs_dh_arr = np.log(dh_wrapper.dhinit_soln['rs'])[::-1]
    logrs = np.log(1+spin_temp.redshift)
    i = np.searchsorted(logrs_dh_arr, logrs)
    logrs_left, logrs_right = logrs_dh_arr[i-1:i+1]

    dh_spec_N_arr = np.array([s.N for s in dh_wrapper.dhinit_soln['highengphot']])[::-1]
    dh_spec_left, dh_spec_right = dh_spec_N_arr[i-1:i+1]
    dh_spec = ( dh_spec_left * np.abs(logrs - logrs_right) + \
                dh_spec_right * np.abs(logrs - logrs_left) ) / np.abs(logrs_right - logrs_left)
    phot_bath_spec = Spectrum(dh_wrapper.photeng, dh_spec, rs=1+spin_temp.redshift, spec_type='N')
else:
    phot_bath_spec = Spectrum(dh_wrapper.photeng, np.zeros_like(photeng),
                              rs=1+spin_temp.redshift, spec_type='N') # [N per Bavg]

## Now that we are synchronized, we enter our loop

In [12]:
records = []
pbar = tqdm(total=len(z_edges)-1, position=0)
 
for i_z in range(len(z_edges)-1):
    dh_wrapper.set_empty_arrays
    
    print()
    print('Starting step: ', i_z)
    
    current_z, next_z, dt = get_time_step(i_z)
    nBavg = phys.n_B * (1+current_z)**3 # [Bavg / (physical cm)^3]
    
    print('Currently at: ', current_z)
    print('Advancing to: ', next_z)
    
    # Calculate the perturbed field at the current time
    perturbed_field = p21c.perturb_field(redshift=current_z, init_boxes=p21c_initial_conditions)
    delta_plus_one_box = 1+ jnp.asarray(perturbed_field.density)

    # Derived quantities that I need
    rho_DM_box = delta_plus_one_box * phys.rho_DM * (1+current_z)**3 # [eV/(physical cm)^3]
    x_e_box = jnp.asarray(1 - ionized_box.xH_box) # check this
    inj_per_Bavg_box = phys.inj_rate(rho_DM_box, dm_params) * dt * struct_boost(1+current_z) / nBavg # [inj/Bavg]
    
    # Kwargs for the transfer function interpolation. I had to put the hacky cap on `x_s`. Look into this
    tf_kwargs = dict(rs = 1 + current_z, nBs_s = delta_plus_one_box.ravel(),
                     x_s = np.minimum(.999, x_e_box).ravel(), 
                     out_of_bounds_action = 'clip')
    
    # Construct the empty arrays we deposit into
    dh_wrapper.set_empty_arrays()   
    
    # Set the kwargs for use in interpolations
    dh_wrapper.set_tf_kwargs(tf_kwargs)
    

    ############################
    ###   X-Ray Deposition   ###
    ############################
        
    # Now calculate photon emission and energy deposition from our X-ray annuli
    for i_z_shell in range(xray_loop_start, i_z):

        # Load the effective density and emission spectrum.
        print(i_z_shell, z_edges[i_z_shell], z_edges[i_z_shell+1])
        xray_out = xray_windowed_data.get_smoothed_shell(current_z, z_edges[i_z_shell], z_edges[i_z_shell+1])
        effective_density, emission_spectrum, is_box_average = xray_out

        # Convert the effective density from photons/(comoving cm)^3 to photon / baryon
        xray_spec = Spectrum(dh_wrapper.photeng, emission_spectrum, rs=z_edges[i_z_shell], spec_type='N') # [photon / Bavg]

        # Attenuate the spectrum
        for j in range(i_z_shell+1, i_z):
            xray_spec.N *= attenuation_arr[j]
            xray_spec.redshift(z_edges[j])     

        # If we are smoothing on the scale of the box then dump to the global bath spectrum.
        # The deposition will happen later, and we will not revisit this shell.
        if is_box_average:
            print('Doing box average')
            phot_bath_spec.N += effective_density[0, 0, 0]*xray_spec.N
            xray_loop_start = max(i_z_shell+1, xray_loop_start)

        else:
            print('Injecting')
            dh_wrapper.photon_injection(xray_spec, bath = False, weight_box = effective_density)

    # In the future, we will need the attenuations. Caculate and cache them here
    dep_tf_at_point = dh_wrapper.phot_dep_tf.point_interp(rs=1+current_z, nBs=1, x=np.mean(x_e_box))
    dep_toteng = np.sum(dep_tf_at_point[:, :4], axis=1)

    attenuation_arr[i_z, :] = 1 - dep_toteng/dh_wrapper.photeng

    # Homogeneous bath injection
    dh_wrapper.photon_injection(phot_bath_spec)
    
    # Dark Matter injection
    dh_wrapper.photon_injection(dm_params.inj_phot_spec, bath =False,
                                weight_box=inj_per_Bavg_box, ots = True)
    
    prop_phot_N, emit_phot_N, dep_box =dh_wrapper.get_state_arrays()
    
    
    #################################################################################
    ###   Split the out spectrum between emitted photons and bath photons, then   ###
    ###   prepare them for use in future steps.                                   ###
    #################################################################################


    # Split the x-ray spectrum into bath and emission
    emit_bath_N, emit_xray_N = split_xray(emit_phot_N, ix_lo, ix_hi)
    out_phot_N = prop_phot_N + emit_bath_N

    # Cache the x-ray boxes for future use
    xray_e_box = dep_box[..., 5] / np.dot(dh_wrapper.photeng, emit_xray_N) # energy / B_avg
    xray_windowed_data.set_field(field=xray_e_box, spec=emit_xray_N, z=current_z)

    # Prepare the bath spectrum for the next step    
    out_phot_spec = Spectrum(dh_wrapper.photeng, out_phot_N, rs=1+current_z, spec_type='N')
    out_phot_spec.redshift(1+next_z)
    phot_bath_spec = out_phot_spec
    
    
    ##############################################################
    ###   Update the Input Boxes and Perform a 21cmFAST Step.  ###
    ##############################################################
    
    # Calculate the perturbed field at the next time
    perturbed_field = p21c.perturb_field(redshift=next_z, init_boxes=p21c_initial_conditions)    
    input_heating, input_ionization, input_jalpha = gen_injection_boxes(next_z, p21c_initial_conditions)
    
    # Populate input heating. [K/Bavg] / [B/Bavg] = [K/B]
    input_heating.input_heating += np.array(2 / (3*phys.kB*(1+x_e_box)) * dep_box[...,3] / delta_plus_one_box)
    
    # Populate input ionization. [1/Bavg] / [B/Bavg] = [1/B]
    input_ionization.input_ionization += np.array((dep_box[...,0] + dep_box[...,1]) / phys.rydberg / delta_plus_one_box)

    #  Populate input lyman alpha
    n_lya = dep_box[...,2] * nBavg / phys.lya_eng # [lya cm^-3]
    dnu_lya = (phys.rydberg - phys.lya_eng) / (2*np.pi*phys.hbar) # [Hz^-1]
    J_lya = n_lya * phys.c / (4*np.pi) / dnu_lya # [lya cm^-2 s^-1 sr^-1 Hz^-1]
    input_jalpha.input_jalpha += np.array(J_lya)
    spin_temp, ionized_box, brightness_temp = p21_step(next_z, perturbed_field, spin_temp, ionized_box,
                                                       input_heating, input_ionization, input_jalpha)
    
    
    #######################################################
    ###   Save some Global Quantities for Convenience   ###
    #######################################################
    
    dE_inj_per_Bavg = dm_params.eng_per_inj * np.mean(inj_per_Bavg_box) # [eV per Bavg]
    dE_inj_per_Bavg_unclustered = dE_inj_per_Bavg / struct_boost(1+current_z)

     
    record_inj = {'dE_inj_per_B' : dE_inj_per_Bavg,
                  'f_ion'  : np.mean(dep_box[...,0] + dep_box[...,1]) / dE_inj_per_Bavg_unclustered,
                  'f_exc'  : np.mean(dep_box[...,2]) / dE_inj_per_Bavg_unclustered,
                  'f_heat' : np.mean(dep_box[...,3]) / dE_inj_per_Bavg_unclustered,
    }
    
    record = {
        'z'   : next_z,
        'T_s' : np.mean(spin_temp.Ts_box), # [mK]
        'T_b' : np.mean(brightness_temp.brightness_temp), # [K]
        'T_k' : np.mean(spin_temp.Tk_box), # [K]
        'x_e' : np.mean(1 - ionized_box.xH_box), # [1]
        'E_phot' : phot_bath_spec.toteng(), # [eV/Bavg]
    }
    if run_mode in ['bath', 'xray']:
        record.update(record_inj)
    records.append(record)
    print(record)
    
    if np.isnan(phot_bath_spec.toteng()):
        break

    pbar.update()
    
    arr_records = {k: np.array([r[k] for r in records]) for k in records[0].keys()}
    np.save('./New_Debug', arr_records)

  0%|          | 0/188 [00:00<?, ?it/s]


Starting step:  0
Currently at:  44
Advancing to:  43.554455445544555


  1%|          | 1/188 [00:05<15:59,  5.13s/it]

{'z': 43.554455445544555, 'T_s': 150.1659, 'T_b': 10.870818, 'T_k': 235.60333, 'x_e': 0.00569528, 'E_phot': Array(9.350296, dtype=float32), 'dE_inj_per_B': Array(0.02933043, dtype=float32), 'f_ion': Array(0.03235668, dtype=float32), 'f_exc': Array(0.02469284, dtype=float32), 'f_heat': Array(0.0271366, dtype=float32)}

Starting step:  1
Currently at:  43.554455445544555
Advancing to:  43.11332222331144
0 44 43.554455445544555
43.554455445544555 44 43.554455445544555 11.744011231769205 0.0
Injecting


/global/scratch/projects/pc_heptheory/fosterjw/21CM_Project/DM21cm/notebooks/../dm21cm/field_smoother.py:160: RuntimeWarning: invalid value encountered in divide
  W1 = 3*(np.sin(self.kMag*R1) - self.kMag*R1 * np.cos(self.kMag*R1)) /(self.kMag*R1)**3
/global/scratch/projects/pc_heptheory/fosterjw/21CM_Project/DM21cm/notebooks/../dm21cm/field_smoother.py:161: RuntimeWarning: invalid value encountered in divide
  W2 = 3*(np.sin(self.kMag*R2) - self.kMag*R2 * np.cos(self.kMag*R2)) /(self.kMag*R2)**3
  1%|          | 2/188 [00:08<13:32,  4.37s/it]

...

{'z': 6.07028613698021, 'T_s': 849.2344, 'T_b': 1.3929844, 'T_k': 1322.3502, 'x_e': 0.8965277, 'E_phot': Array(245.26456, dtype=float32), 'dE_inj_per_B': Array(5.429436, dtype=float32), 'f_ion': Array(0.03694892, dtype=float32), 'f_exc': Array(0.01091884, dtype=float32), 'f_heat': Array(0.5359005, dtype=float32)}

Starting step:  186
Currently at:  6.07028613698021
Advancing to:  6.000283303940802
183 6.2845228772168475 6.212398888333512
6.07028613698021 6.2845228772168475 6.212398888333512 88.17915647669358 58.92989499392185
Doing box average
184 6.212398888333512 6.140988998350012
6.07028613698021 6.212398888333512 6.140988998350012 58.92989499392185 29.53705092785163
Injecting
185 6.140988998350012 6.07028613698021
6.07028613698021 6.140988998350012 6.07028613698021 29.53705092785163 0.0
Injecting


 99%|█████████▉| 187/188 [20:24<00:05,  5.15s/it]

{'z': 6.000283303940802, 'T_s': 861.59814, 'T_b': 1.2048032, 'T_k': 1363.0198, 'x_e': 0.90864044, 'E_phot': Array(248.25685, dtype=float32), 'dE_inj_per_B': Array(5.4755607, dtype=float32), 'f_ion': Array(0.03693602, dtype=float32), 'f_exc': Array(0.01065393, dtype=float32), 'f_heat': Array(0.54270446, dtype=float32)}

Starting step:  187
Currently at:  6.000283303940802
Advancing to:  5.93097356825822
184 6.212398888333512 6.140988998350012
6.000283303940802 6.212398888333512 6.140988998350012 88.6117769183169 59.21893290634641
Doing box average
185 6.140988998350012 6.07028613698021
6.000283303940802 6.140988998350012 6.07028613698021 59.21893290634641 29.681881996475784
Injecting
186 6.07028613698021 6.000283303940802
6.000283303940802 6.07028613698021 6.000283303940802 29.681881996475784 0.0
Injecting


100%|██████████| 188/188 [20:30<00:00,  5.43s/it]

{'z': 5.93097356825822, 'T_s': 872.9759, 'T_b': 1.0386764, 'T_k': 1403.395, 'x_e': 0.919605, 'E_phot': Array(251.26547, dtype=float32), 'dE_inj_per_B': Array(5.521962, dtype=float32), 'f_ion': Array(0.03701083, dtype=float32), 'f_exc': Array(0.01045546, dtype=float32), 'f_heat': Array(0.5489898, dtype=float32)}


In [ ]:
lightcone = p21c.run_lightcone(redshift = ionized_box.redshift,
                               user_params = ionized_box.user_params,
                               cosmo_params = ionized_box.cosmo_params,
                               astro_params = ionized_box.astro_params,
                               flag_options=ionized_box.flag_options, )

/global/scratch/projects/pc_heptheory/fosterjw/21CM_Project/21cmFAST/src/py21cmfast/_utils.py:821: UserWarning: Trying to remove array that isn't yet created: hires_vx
  warnings.warn(f"Trying to remove array that isn't yet created: {k}")
/global/scratch/projects/pc_heptheory/fosterjw/21CM_Project/21cmFAST/src/py21cmfast/_utils.py:821: UserWarning: Trying to remove array that isn't yet created: hires_vy
  warnings.warn(f"Trying to remove array that isn't yet created: {k}")
/global/scratch/projects/pc_heptheory/fosterjw/21CM_Project/21cmFAST/src/py21cmfast/_utils.py:821: UserWarning: Trying to remove array that isn't yet created: hires_vz
  warnings.warn(f"Trying to remove array that isn't yet created: {k}")
/global/scratch/projects/pc_heptheory/fosterjw/21CM_Project/21cmFAST/src/py21cmfast/_utils.py:821: UserWarning: Trying to remove array that isn't yet created: hires_vx_2LPT
  warnings.warn(f"Trying to remove array that isn't yet created: {k}")
/global/scratch/projects/pc_heptheory/f

In [ ]:
from py21cmfast import plotting

In [ ]:
fig, ax = plt.subplots(figsize = (14, 10))
plotting.lightcone_sliceplot(lightcone, fig =fig,ax = ax) 

In [ ]:
def compute_power(
   box,
   length,
   n_psbins,
   log_bins=True,
   ignore_kperp_zero=True,
   ignore_kpar_zero=False,
   ignore_k_zero=False,
):
    # Determine the weighting function required from ignoring k's.
    k_weights = np.ones(box.shape, int)
    n0 = k_weights.shape[0]
    n1 = k_weights.shape[-1]

    if ignore_kperp_zero:
        k_weights[n0 // 2, n0 // 2, :] = 0
    if ignore_kpar_zero:
        k_weights[:, :, n1 // 2] = 0
    if ignore_k_zero:
        k_weights[n0 // 2, n0 // 2, n1 // 2] = 0

    res = powerbox.tools.get_power(
        box,
        boxlength=length,
        bins=n_psbins,
        bin_ave=False,
        get_variance=False,
        log_bins=log_bins,
        k_weights=k_weights,
    )

    res = list(res)
    k = res[1]
    if log_bins:
        k = np.exp((np.log(k[1:]) + np.log(k[:-1])) / 2)
    else:
        k = (k[1:] + k[:-1]) / 2

    res[1] = k
    return res

def powerspectra(brightness_temp, n_psbins=50, nchunks=20, min_k=0.1, max_k=1.0, logk=True):
    data = []
    chunk_indices = list(range(0,brightness_temp.n_slices,round(brightness_temp.n_slices / nchunks),))    
    
    if len(chunk_indices) > nchunks:
        chunk_indices = chunk_indices[:-1]
    chunk_indices.append(brightness_temp.n_slices)

    for i in range(nchunks):
        start = chunk_indices[i]
        end = chunk_indices[i + 1]
        chunklen = (end - start) * brightness_temp.cell_size

        power, k = compute_power(
            brightness_temp.brightness_temp[:, :, start:end],
            (BOX_LEN, BOX_LEN, chunklen),
            n_psbins,
            log_bins=logk,
        )
        data.append({"k": k, "delta": power * k ** 3 / (2 * np.pi ** 2)})
    return data

In [ ]:
k_fundamental = 2*np.pi/BOX_LEN
k_max = k_fundamental * HII_DIM
Nk=np.floor(HII_DIM/1).astype(int)

In [ ]:
out = powerspectra(lightcone, min_k = k_fundamental, max_k = k_max)


In [ ]:
fig, axs = plt.subplots(ncols = 5, nrows = 4, figsize = (30, 16))

for i, item in enumerate(out):
    row_index, col_index = np.unravel_index(i, axs.shape)
    
    ax = axs[row_index, col_index]
    ax.plot(item['k'], item['delta'], color = 'black')
    ax.set_xscale('log')
    ax.set_yscale('log')
    ax.text(.6, .1, 'Redshift Chunk ' + str(i), transform=ax.transAxes, fontsize = 18)
    
for i in range(axs.shape[0]-1):
    for j in range(axs.shape[1]):
        axs[i, j].set_xticklabels([])

for j in range(axs.shape[1]):
    axs[-1, j].set_xlabel('k [Mpc$^{-1}$]', fontsize = 22)
for i in range(axs.shape[0]):
    axs[i, 0].set_ylabel('$k^3 P(k)$', fontsize = 22)
    
plt.tight_layout()
plt.show()